In [1]:
import numpy as np

In [2]:
with open('data/profile_intro.csv') as f:
    data = f.read().splitlines()

In [3]:
string = ''.join(data)
char_to_ind = {c: i for i, c in enumerate(sorted(set(string)))}
char_to_ind[''] = -99

ind_to_char = {v: k for k, v in char_to_ind.items()}

In [4]:
chars = np.array([np.array([char_to_ind[c] for c in line] + [-99]) for line in data])

In [5]:
def generate_batches(array, batch_size=100, num_char=10):
    i = 0
    array = np.concatenate(array)
    while True:
        x = []
        y = []
        for _ in range(batch_size):
            # Slice x
            start = i
            end = i + num_char
            x.append(array[start:end])
            
            # Slice y
            start += 1
            end += 1
            y.append(array[start:end])
            
            if end >= len(array):
                i = 0
            else:
                i += 1

        yield np.array(x), np.array(y)
        

batches = generate_batches(chars)

# for i in range(500000):
for i in range(100):
    x, y = next(batches)

In [6]:
from char_rnn import CharRNN

/Users/Ikkei/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [12]:
# these are preset parameters, you can change them to get better result
batch_size = 100         # Sequences per batch
num_steps = 100          # Number of sequence steps per batch
rnn_size = 256           # Size of hidden layers in rnn_cell
num_layers = 2           # Number of hidden layers
learning_rate = 0.005    # Learning rate

In [13]:
model = CharRNN(len(char_to_ind), batch_size, num_steps, 'LSTM', rnn_size,
                num_layers, learning_rate)
batches = generate_batches(chars, batch_size, num_steps)
model.train(batches, max_count=9000, save_every_n=3000)

step: 200  loss: 2.7516  0.8889 sec/batch


KeyboardInterrupt: 